# 인스타그램 - 제주맛집

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [2]:
import unicodedata, re
from tqdm.notebook import tqdm

In [3]:
insta_url = 'https://www.instagram.com'
driver = webdriver.Chrome('chromedriver')
driver.get(insta_url)
time.sleep(1)

### 1. 로그인

In [4]:
with open('password.txt') as f:
    password = f.read()

In [5]:
email = 'higoodgood'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [6]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [7]:
# 로그인 정보 저장 - 나중에 하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [8]:
# 알림 설정 - 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 2. 제주맛집으로 검색

In [9]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [10]:
driver.get(url)
time.sleep(5)

## 3. 첫번째 게시글

In [11]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [12]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [13]:
# 1) 본문 내용 가져오기

try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'영업시간 네이버 참고      #제주맛집 #제주도맛집#에코랜드 #사려니숲길 #코코몽에코파크제주 #스누피가든 #주상절리 #제주아쿠아플라넷 #안돌오름 #제주동쪽숙소 #제주서쪽숙소 #제주서쪽여행 #제주동쪽여행 #카멜리아힐 #성이시돌목장 #제주신상카페 #금오름 #빛의벙커 #아르떼뮤지엄#버버리제주#용머리해안'

In [14]:
# 2) 본문 내용에서 해쉬태그 가져오기, '#[^\s#,\\]+'
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#제주맛집',
 '#제주도맛집',
 '#에코랜드',
 '#사려니숲길',
 '#코코몽에코파크제주',
 '#스누피가든',
 '#주상절리',
 '#제주아쿠아플라넷',
 '#안돌오름',
 '#제주동쪽숙소',
 '#제주서쪽숙소',
 '#제주서쪽여행',
 '#제주동쪽여행',
 '#카멜리아힐',
 '#성이시돌목장',
 '#제주신상카페',
 '#금오름',
 '#빛의벙커',
 '#아르떼뮤지엄',
 '#버버리제주',
 '#용머리해안']

In [15]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-12-07'

In [16]:
# 4) 좋아요 수 가져오기: 좋아요 1,296개
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

'140'

In [17]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

''

In [18]:
row = [content, date, like, place, tags]
row

['영업시간 네이버 참고      #제주맛집 #제주도맛집#에코랜드 #사려니숲길 #코코몽에코파크제주 #스누피가든 #주상절리 #제주아쿠아플라넷 #안돌오름 #제주동쪽숙소 #제주서쪽숙소 #제주서쪽여행 #제주동쪽여행 #카멜리아힐 #성이시돌목장 #제주신상카페 #금오름 #빛의벙커 #아르떼뮤지엄#버버리제주#용머리해안',
 '2021-12-07',
 '140',
 '',
 ['#제주맛집',
  '#제주도맛집',
  '#에코랜드',
  '#사려니숲길',
  '#코코몽에코파크제주',
  '#스누피가든',
  '#주상절리',
  '#제주아쿠아플라넷',
  '#안돌오름',
  '#제주동쪽숙소',
  '#제주서쪽숙소',
  '#제주서쪽여행',
  '#제주동쪽여행',
  '#카멜리아힐',
  '#성이시돌목장',
  '#제주신상카페',
  '#금오름',
  '#빛의벙커',
  '#아르떼뮤지엄',
  '#버버리제주',
  '#용머리해안']]

## 4. 다음 게시글로 이동

In [19]:
# 첫번째 게시글에서 다음글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button').click()
time.sleep(2)

In [20]:
# 두번째 게시글부터 다음글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()
time.sleep(2)

In [21]:
# 5를 하기 위해 popup 화면을 없앰(즉, X 를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

### 5. 반복문으로 데이터 가져오기

In [22]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]

In [23]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [24]:
results = []
for _ in tqdm(range(500)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()
        time.sleep(3)
    except:
        time.sleep(3)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button').click()
        time.sleep(3)

  0%|          | 0/500 [00:00<?, ?it/s]

In [25]:
driver.close()

### 6. 데이터프레임으로 만들고 중복 제거

In [26]:
df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
df.head()

,content,date,like,place,tags
0,영업시간 네이버 참고 #제주맛집 #제주도맛집#에코랜드 #사려니숲길 #코코몽...,2021-12-07,140,,"[#제주맛집, #제주도맛집, #에코랜드, #사려니숲길, #코코몽에코파크제주, #스누..."
1,".❤제주 ""제이쁠쁠""❤제주 어디든 예약만 해두면 숙소로배달와버리는 제주출신흑돼지!!...",2021-12-07,"1,075",,"[#제주맛집, #제주도맛집, #제주공항근처맛집, #제주시맛집, #애월맛집, #중문맛..."
2,"협찬인기개그맨 옥동자 정종철이 강추하는제주맛집 ""랍스터빈""주소 : 제주시 애월읍 애...",2021-12-07,i_m._dd_._d님 여러 명이 좋아합니,랍스터빈_jeju,"[#애월맛집, #애월카페, #제주애월맛집, #제주도맛집, #제주맛집, #제주도카페,..."
3,"아침부터 스튜 포장이 많네요🙋‍♀️스튜는 마늘빵과 함께 제공해드리고 있으며,마늘빵 ...",2021-12-08,447,월정리해변,[#안녕하샌]
4,#리그램 - @gunwoo_o942021. 11. 30 제주 1일❤️..또 가고싶다...,2021-12-07,333,월정리해변,"[#리그램, #안녕하샌]"


In [27]:
df.drop_duplicates(subset='content', keep='first', inplace=True)
df.shape

(330, 5)

In [28]:
df.to_csv('인스타-제주맛집.csv', index=False)